SmartParkingSystem

In [8]:
from supervision import BoxAnnotator


In [9]:
import cv2
import torch
import easyocr
import numpy as np
from ultralytics import YOLO
from supervision.geometry.core import Point
from supervision.annotators.core import BoxAnnotator
from supervision.detection.core import Detections

from supervision.detection.line_zone import LineZone, LineZoneAnnotator
from supervision.annotators.core import LabelAnnotator

# from supervision.tracker.byte_track.byte_track import ByteTrack



from datetime import datetime, timedelta
import math
import time
import argparse # For command line arguments

Configuration

In [36]:
import argparse
import sys

# Use argparse for command line arguments
parser = argparse.ArgumentParser(description="Smart Parking System with YOLOv8, EasyOCR, and Tracking")
parser.add_argument("--video", default="video.mp4", help="Path to the input video file.")
parser.add_argument("--vehicle-model", default="yolov8n.pt", help="Path to the YOLOv8 vehicle detection model.")
parser.add_argument("--plate-model", default=None, help="Path to the YOLOv8 license plate detection model (optional). If not provided, a heuristic crop is used.")
parser.add_argument("--hourly-rate", type=float, default=20.0, help="Hourly parking rate.")
parser.add_argument("--confidence", type=float, default=0.3, help="Detection confidence threshold.")
parser.add_argument("--iou", type=float, default=0.5, help="Detection IOU threshold.")
parser.add_argument("--line-y", type=int, default=700, help="Y-coordinate for the horizontal entry/exit line.")
parser.add_argument("--entry-direction", default="down", choices=["up", "down"], help="Direction crossing the line that signifies entry ('up' or 'down').")

# Fix: avoid error from extra Jupyter arguments
args, _ = parser.parse_known_args()


VIDEO_PATH = args.video
VEHICLE_MODEL_PATH = args.vehicle_model
PLATE_MODEL_PATH = args.plate_model
HOURLY_RATE = args.hourly_rate
CONFIDENCE_THRESHOLD = args.confidence
IOU_THRESHOLD = args.iou
LINE_Y = args.line_y # Y-coordinate of the horizontal line
ENTRY_DIRECTION_IS_DOWN = (args.entry_direction == "down") # True if crossing downwards is entry



Entry-Exit Line

In [37]:
# Define the horizontal line across the full width
# We'll get frame width later to make it dynamic
LINE_START = Point(0, LINE_Y)
LINE_END = Point(0, LINE_Y) # Will update X coordinate after reading first frame

Initialize the Model and readers

In [38]:
# --- Model and Reader Initialization ---
print("Loading models and OCR...")
try:
    vehicle_model = YOLO(VEHICLE_MODEL_PATH)

    plate_model = None
    USE_SEPARATE_PLATE_MODEL = False
    if PLATE_MODEL_PATH and PLATE_MODEL_PATH.lower() != 'none':
        try:
            plate_model = YOLO(PLATE_MODEL_PATH)
            USE_SEPARATE_PLATE_MODEL = True
            print(f"Using separate plate detection model: {PLATE_MODEL_PATH}")
        except Exception as e:
            print(f"Warning: Could not load plate model from {PLATE_MODEL_PATH}: {e}. Falling back to heuristic crop.")
            PLATE_MODEL_PATH = None # Ensure we don't keep trying to load it

    if not USE_SEPARATE_PLATE_MODEL:
        print("No separate plate model provided or failed to load. Using heuristic crop from vehicle bounding box for OCR.")
        print("This method is less reliable. Consider training or providing a license plate YOLO model.")


    # Initialize EasyOCR reader
    # This can take some time on the first run as it downloads models
    reader = easyocr.Reader(['en']) # Use english language model
    print("Models and OCR loaded successfully.")

except Exception as e:
    print(f"Error loading models or EasyOCR: {e}")
    print("Please ensure YOLOv8 models are in the correct path and EasyOCR dependencies are met.")
    exit()


Loading models and OCR...
No separate plate model provided or failed to load. Using heuristic crop from vehicle bounding box for OCR.
This method is less reliable. Consider training or providing a license plate YOLO model.
Models and OCR loaded successfully.


Initializing the Tracking

In [ ]:
# --- Tracking Initialization ---
# ByteTrack helps maintain consistent IDs for vehicles across frames
try:
    tracker = ByteTrack(frame_rate=30) # Adjust frame_rate if known
    print("ByteTrack tracker initialized.")
except ImportError:
    print("Error: ByteTrack not found. Please install it using 'pip install bytetracker' or 'pip install supervision[tracker]'. Exiting.")
    exit()
except Exception as e:
     print(f"Error initializing ByteTrack: {e}. Exiting.")
     exit()

Error initializing ByteTrack: name 'ByteTrack' is not defined. Exiting.


: 